<a href="https://colab.research.google.com/github/ronanmccormack-ca/Group-C-Machine-Learning/blob/main/voting_classifier_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import matplotlib.pyplot as plt 
import numpy as np
import os 
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

Using this block to read in my files

In [18]:
df_peugeot_1 = pd.read_csv('peugeot_207_01.csv',sep=";",index_col=0)
df_peugeot_2 = pd.read_csv('peugeot_207_02.csv',sep=";",index_col=0)
df_corsa_1 = pd.read_csv('opel_corsa_01.csv',sep=";",index_col=0)
df_corsa_2 = pd.read_csv('opel_corsa_02.csv',sep=";",index_col=0)

In [19]:
df_peugeot_1 = pd.read_csv("C:\\Users\\stjohns\\Downloads\\opel_corsa_01 (1).csv",sep=";")
df_peugeot_2 = pd.read_csv("C:\\Users\\stjohns\\Downloads\\opel_corsa_02.csv",sep=";")
df_corsa_1 = pd.read_csv("C:\\Users\\stjohns\\Downloads\\peugeot_207_01.csv",sep=";")
df_corsa_2 = pd.read_csv("C:\\Users\\stjohns\\Downloads\\peugeot_207_02.csv",sep=";")

FileNotFoundError: ignored

Merging the two datasets for each vehicle.

In [20]:
df_peugeot = pd.concat([df_peugeot_1,df_peugeot_2])
df_corsa = pd.concat([df_corsa_1,df_corsa_2])

Dropping the NaN values from each dataset and resetting the index.

In [21]:
df_peugeot = df_peugeot.dropna()
df_corsa = df_corsa.dropna()

df_peugeot = df_peugeot.reset_index(drop=True)
df_corsa = df_corsa.reset_index(drop=True)

In [22]:
df_peugeot.head()

,AltitudeVariation,VehicleSpeedInstantaneous,VehicleSpeedAverage,VehicleSpeedVariance,VehicleSpeedVariation,LongitudinalAcceleration,EngineLoad,EngineCoolantTemperature,ManifoldAbsolutePressure,EngineRPM,MassAirFlow,IntakeAirTemperature,VerticalAcceleration,FuelConsumptionAverage,roadSurface,traffic,drivingStyle
0,-13.399994,20.699999,6.060,115.840061,-4.500000,1.0548,38.039215,31.0,101.0,798.0,6.22,15.0,-0.5982,27.919697,SmoothCondition,LowCongestionCondition,EvenPaceStyle
1,-13.200005,17.100000,6.345,117.200816,-3.599998,0.9012,38.039215,31.0,101.0,797.5,6.27,15.0,-0.5244,26.458355,SmoothCondition,LowCongestionCondition,EvenPaceStyle
2,-13.200005,14.400000,6.585,117.559595,-2.700001,0.9331,90.588234,32.0,101.0,797.5,6.27,15.0,-0.5068,26.458355,SmoothCondition,LowCongestionCondition,EvenPaceStyle
3,-11.800003,14.400000,6.825,117.801222,0.000000,0.9878,90.588234,32.0,101.0,726.0,8.63,15.0,-0.4882,26.004126,SmoothCondition,LowCongestionCondition,EvenPaceStyle
4,-19.800003,14.400000,7.065,117.925697,0.000000,1.1316,92.941177,32.0,101.0,784.0,8.88,15.0,-0.6712,25.295498,SmoothCondition,LowCongestionCondition,EvenPaceStyle


Making sure the Dtypes are of numerical and non-numerical.

In [23]:
df_peugeot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12632 entries, 0 to 12631
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   AltitudeVariation          12632 non-null  float64
 1   VehicleSpeedInstantaneous  12632 non-null  float64
 2   VehicleSpeedAverage        12632 non-null  float64
 3   VehicleSpeedVariance       12632 non-null  float64
 4   VehicleSpeedVariation      12632 non-null  float64
 5   LongitudinalAcceleration   12632 non-null  float64
 6   EngineLoad                 12632 non-null  float64
 7   EngineCoolantTemperature   12632 non-null  float64
 8   ManifoldAbsolutePressure   12632 non-null  float64
 9   EngineRPM                  12632 non-null  float64
 10  MassAirFlow                12632 non-null  float64
 11  IntakeAirTemperature       12632 non-null  float64
 12  VerticalAcceleration       12632 non-null  float64
 13  FuelConsumptionAverage     12632 non-null  flo

Inviting the models to the party.

In [24]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Splitting the X set and y set. Traditionally y is one column but because we want to analyse the optimal output we include all 3 into y and iterate through them at a later stage. As y is now a matrix we should technically use a Y to decribe it but we won't worry about it.

In [25]:
y = df_peugeot.select_dtypes(object)
X = df_peugeot.drop(columns=df_peugeot.select_dtypes(object))

#X=df_peugeot.iloc[:,0:100]
#y=df_peugeot.iloc[:,100]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Turning the voting classifier into a function so we can call it at anytime for any variables without changing the actual model. Inputting X,y and the vote (hard or soft) and returning train & test score.

In [31]:
def voting_class(X,y,vote):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    model = VotingClassifier([
    ('nb', GaussianNB()),
    ('knn', KNeighborsClassifier(5)),
    ('svm', SVC(kernel='linear', C=0.1)),
    ('tree1', DecisionTreeClassifier(max_depth=4)),
    ('tree2', DecisionTreeClassifier(min_samples_leaf=10))],
    voting=vote)
    model.fit(X_train , y_train)
    train_score = model.score(X_train , y_train)
    test_score = model.score(X_test , y_test)
    return train_score,test_score

Iteration I talked about earlier. Iterate through each y column while keeping the same X matrix for prediction. Takes longer than just having 1 y values but we want to find the optimal output. Saving the returned values into variables and printing them at the end of each loop.

In [32]:
for x in y.columns:
    z=y[x]
    train_score,test_score = voting_class(X,z,'hard')
    print(train_score,test_score)

0.8659489128140173 0.8499050031665611


KeyboardInterrupt: ignored

In [71]:
y = y['roadSurface']

In [72]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from matplotlib import pyplot

In [73]:
def get_voting():
	# define the base models
	models = list()
	models.append(('nb', GaussianNB()))
	models.append(('knn', KNeighborsClassifier(5)))
	models.append(('svm', SVC(kernel='linear', C=0.1)))
	models.append(('tree1', DecisionTreeClassifier(max_depth=4)))
	models.append(('tree2', DecisionTreeClassifier(min_samples_leaf=10)))
	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='hard')
	return ensemble

In [74]:
def get_models():
	models = dict()
	models['nb'] = GaussianNB()
	models['knn'] = KNeighborsClassifier(n_neighbors=5)
	models['svc'] = SVC(kernel='linear', C=0.1)
	models['tree1'] = DecisionTreeClassifier(max_depth=4)
	models['tree2'] = DecisionTreeClassifier(min_samples_leaf=10)
	models['hard_voting'] = get_voting()
	return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores
 
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

>nb 0.736 (0.013)
>knn 0.823 (0.011)


KeyboardInterrupt: ignored

In [ ]:
y